In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt


In [ ]:
# Load the dataset
file_path = 'Sample_Longlist_Data.xlsx'
data = pd.read_excel(file_path)

In [ ]:
# Display basic information about the dataset
print("Dataset Information:")
data.info()

In [ ]:
# Display first few rows to understand the structure
print("\nFirst 5 rows of the dataset:")
print(data.head())

In [ ]:
# Check column names
print("\nColumn names:")
print(data.columns)

In [ ]:
# Data Sanitization
# 1. Parse product counts
def parse_product_count(product_count_str):
    match = re.search(r'over (\d+[,.\d+]*)', str(product_count_str))
    return int(match.group(1).replace(',', '')) if match else 0

data['parsed_product_count'] = data['sellerproductcount'].apply(parse_product_count)


In [ ]:
# 2. Extract ratings percentage and count
def parse_seller_ratings(rating_str):
    percentage_match = re.search(r'(\d+)%', str(rating_str))
    count_match = re.search(r'\((\d+) ratings\)', str(rating_str))
    percentage = int(percentage_match.group(1)) if percentage_match else 0
    count = int(count_match.group(1)) if count_match else 0
    return percentage, count

ratings = data['sellerratings'].apply(parse_seller_ratings)
data['rating_percentage'] = ratings.apply(lambda x: x[0])
data['rating_count'] = ratings.apply(lambda x: x[1])


In [ ]:
# 3. Extract phone numbers and emails
data['phone_number'] = data['sellerdetails'].str.extract(r'(\+?\d{1,4}[\s\-]?\d{1,4}[\s\-]?\d{4,})')
data['email'] = data['sellerdetails'].str.extract(r'([\w\.-]+@[\w\.-]+)')

In [ ]:
# 4. Parse business address for country
def parse_country(address):
    if 'US' in str(address):
        return 'US'
    elif 'DE' in str(address):
        return 'Germany'
    elif 'CN' in str(address):
        return 'China'
    else:
        return 'Other'

data['country'] = data['businessaddress'].apply(parse_country)


In [ ]:
# Filter out Chinese sellers
data = data[data['country'] != 'China']


In [ ]:
# Selection Criteria
best_sellers = data[(data['parsed_product_count'] > 1000) & 
                    (data['rating_percentage'] > 80) & 
                    (data['rating_count'] > 100) & 
                    (data['country'].isin(['US', 'Germany']))]


In [ ]:
# Check for correct seller name column
seller_name_column = 'sellername' if 'sellername' in best_sellers.columns else best_sellers.columns[0]

In [ ]:
# Display best sellers
print("\nBest Sellers:")
print(best_sellers[[seller_name_column, 'parsed_product_count', 'rating_percentage', 'rating_count', 'country', 'phone_number', 'email']])


In [ ]:
# Save best sellers to Excel
output_file = 'Best_Sellers2.xlsx'
best_sellers.to_excel(output_file, index=False)

print(f"\nBest sellers data has been saved to: {output_file}")

In [ ]:
# Visualization
plt.figure(figsize=(10, 6))
best_sellers['country'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Number of Best Sellers by Country')
plt.xlabel('Country')
plt.ylabel('Number of Sellers')
plt.show()